In [5]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

Using TensorFlow backend.


In [6]:
df = pd.read_csv('train.csv')

In [9]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,target
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,False
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,False
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,False
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,False
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,False


In [7]:
df['target'] = (df.toxic + df.severe_toxic + df.obscene + df.threat + df.insult + df.identity_hate) > 0

In [8]:
y = pd.get_dummies(df.target).values

In [11]:
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(df.comment_text))
list_tokenized_headlines = tokenizer.texts_to_sequences(df.comment_text)
X_t = sequence.pad_sequences(list_tokenized_headlines, maxlen=100)

In [17]:
embedding_size = 128
input_ = Input(shape=(100,))
x = Embedding(20000, embedding_size)(input_)
x = LSTM(25, return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.5)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='sigmoid')(x)

model = Model(inputs=input_, outputs=x)

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 100, 128)          2560000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 25)           15400     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 25)                0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                1300      
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
__________

In [20]:
model.fit(X_t, y, epochs=2, batch_size=32, validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 595s 4ms/step - loss: 0.1551 - acc: 0.9488 - val_loss: 0.1078 - val_acc: 0.9588
Epoch 2/2
143613/143613 [==============================] - 590s 4ms/step - loss: 0.1037 - acc: 0.9636 - val_loss: 0.1079 - val_acc: 0.9621


In [21]:
from sklearn.metrics import classification_report

In [44]:
#y_preds = model.predict(X_t)
y_preds =[x[0] for x in y_preds]
y_preds = [int(round(x)) for x in y_preds]
y = [x[0] for x in y]

In [46]:
print(classification_report(y_preds, y))

              precision    recall  f1-score   support

           0       0.82      0.90      0.86     14715
           1       0.99      0.98      0.98    144856

   micro avg       0.97      0.97      0.97    159571
   macro avg       0.90      0.94      0.92    159571
weighted avg       0.97      0.97      0.97    159571

